In [48]:
import os
import pandas as pd
import pickle
from typing import List, Union
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

class DataPrep:

    df: pd.DataFrame
    metadata: str
    col_names: List[str]
    cols_to_scale: List[str] = ['age', 'wage per hour', 'dividends from stocks', 'weeks worked in year', 'capital_change']
    cols_to_encode: List[str] = ['class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'year']

    def __init__(self, data_path: str):
        self.data_path = data_path
        self.inhale_metadata()

    def create_dataframe(self, print_=False):
        self.df = pd.read_csv(self.data_path, header=None)
        if print_:
            print(self.df.head(3))

    def inhale_metadata(self):
        with open('./census_data/census_income_metadata.txt', 'r') as f:
            self.metadata = f.read()

    def get_col_names(self, print_=False):
        self.col_names = [attr.split(':')[0] for attr in self.metadata.split('- 50000, 50000+.')[1].strip().split('\n')]
        self.col_names.remove('| instance weight'), self.col_names.append('target')
        if print_:
            print(self.col_names)

    def set_col_names(self, print_=False):
        self.df.columns = self.col_names
        if print_:
            print(self.df.head())

    def feature_engineering(self):
        """metadata indicated this feature should be dropped"""
        self.df.drop('instance weight', axis=1, inplace=True)

        """a single individual can't have both a capital gain AND a capital loss so condense those"""
        self.df['capital_change'] = self.df['capital gains'] - self.df['capital losses']
        self.df.drop(['capital gains', 'capital losses'], axis=1, inplace=True)

        """replace the target feature string values with integers"""
        self.df['target'].replace({' - 50000.': -1, ' 50000+.': 1}, inplace=True)
        
    def numerical_scaling(self):
        data_to_scale = self.df[self.cols_to_scale]
        scaler = StandardScaler()
        data_to_scale = scaler.fit_transform(self.df[self.cols_to_scale])
        self.df[self.cols_to_scale] = data_to_scale

    def categorical_encoding(self, scheme: str = 'target'):
        if scheme == 'label':
            encoder = LabelEncoder()
            label_encoded = pd.DataFrame(columns=self.cols_to_encode)
            for col in self.cols_to_encode:
                label_encoded[col] = encoder.fit_transform(self.df[col])
            self.df[self.cols_to_encode] = label_encoded

        elif scheme == 'target':
            encoder = TargetEncoder(cols=self.cols_to_encode)
            encoder.fit(X=self.df[self.cols_to_encode], y=self.df['target'])
            self.df[self.cols_to_encode] = encoder.transform(self.df[self.cols_to_encode])

        else:
            print('Scheme input incorrect. Please input either "label" or "target"')
    
    def get_val_counts(self, cols: List[str]=None, top_n: int = 8):
        if cols is None: cols = self.df.columns
        for col in cols:
                print(f'{col}: \n{self.df[col].value_counts().iloc[:top_n]}\n')

    def examine_cardinality(self, cols: List[str]=None):
        if cols is None: cols = [col for col in self.df.columns if self.df[col].dtypes!='int64']
        cardinality_series = pd.Series(index=cols)
        for col in cols:
            cardinality_series.loc[col] = self.df[col].nunique()
        print(cardinality_series.sort_values(ascending=False))

    def split_X_y(self, phase: str):
        setattr(self, f'y_{phase}', self.df['target'])
        setattr(self, f'X_{phase}', self.df.drop('target', axis=1))

    def apply_all(self, phase: str, encoding_scheme: str = 'target'):  #####TODO: add all methods to this method
        self.create_dataframe()
        if self.metadata is None: self.inhale_metadata()
        self.get_col_names()
        self.set_col_names()
        self.feature_engineering()
        self.numerical_scaling()
        self.categorical_encoding(encoding_scheme)
        self.split_X_y(phase)
        self.pickle_data(phase)

    def pickle_data(self, phase: str):
        if 'pickles' not in os.listdir(os.getcwd()):
            os.mkdir('./pickles')

        for df in ['X', 'y']:
            with open(f'./pickles/{df}_{phase}.obj', 'wb') as f:
                pickle.dump(getattr(self, f'{df}_{phase}'), f)

In [49]:
train_data = DataPrep('./census_data/census_income_learn.csv')

In [50]:
train_data.create_dataframe(print_=True)

   0                                1   2   3                            4   \
0  73                  Not in universe   0   0         High school graduate   
1  58   Self-employed-not incorporated   4  34   Some college but no degree   
2  18                  Not in universe   0   0                   10th grade   

   5                 6               7                             8   \
0   0   Not in universe         Widowed   Not in universe or children   
1   0   Not in universe        Divorced                  Construction   
2   0       High school   Never married   Not in universe or children   

                                     9   ...              32              33  \
0                       Not in universe  ...   United-States   United-States   
1   Precision production craft & repair  ...   United-States   United-States   
2                       Not in universe  ...         Vietnam         Vietnam   

               34                                    35 36           

In [51]:
train_data.inhale_metadata()
train_data.get_col_names(print_=True)

['age', 'class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'wage per hour', 'enroll in edu inst last wk', 'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 'full or part time employment stat', 'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'region of previous residence', 'state of previous residence', 'detailed household and family stat', 'detailed household summary in household', 'instance weight', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg', 'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer', 'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self', 'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits', 'week

In [52]:
train_data.set_col_names(print_=True)

   age                  class of worker  detailed industry recode  \
0   73                  Not in universe                         0   
1   58   Self-employed-not incorporated                         4   
2   18                  Not in universe                         0   
3    9                  Not in universe                         0   
4   10                  Not in universe                         0   

   detailed occupation recode                    education  wage per hour  \
0                           0         High school graduate              0   
1                          34   Some college but no degree              0   
2                           0                   10th grade              0   
3                           0                     Children              0   
4                           0                     Children              0   

  enroll in edu inst last wk    marital stat           major industry code  \
0            Not in universe         Widowed

In [53]:
train_data.feature_engineering()

In [54]:
train_data.get_val_counts()

age: 
34    3489
35    3450
36    3353
31    3351
33    3340
5     3332
4     3318
3     3279
Name: age, dtype: int64

class of worker: 
 Not in universe                   100245
 Private                            72028
 Self-employed-not incorporated      8445
 Local government                    7784
 State government                    4227
 Self-employed-incorporated          3265
 Federal government                  2925
 Never worked                         439
Name: class of worker, dtype: int64

detailed industry recode: 
0     100684
33     17070
43      8283
4       5984
42      4683
45      4482
29      4209
37      4022
Name: detailed industry recode, dtype: int64

detailed occupation recode: 
0     100684
2       8756
26      7887
19      5413
29      5105
36      4145
34      4025
10      3683
Name: detailed occupation recode, dtype: int64

education: 
 High school graduate                      48407
 Children                                  47422
 Some college but no d

In [55]:
train_data.examine_cardinality()

state of previous residence                   51.0
country of birth self                         43.0
country of birth mother                       43.0
country of birth father                       43.0
detailed household and family stat            38.0
major industry code                           24.0
education                                     17.0
major occupation code                         15.0
migration code-move within reg                10.0
hispanic origin                               10.0
migration code-change in msa                  10.0
class of worker                                9.0
migration code-change in reg                   9.0
detailed household summary in household        8.0
full or part time employment stat              8.0
marital stat                                   7.0
reason for unemployment                        6.0
region of previous residence                   6.0
tax filer stat                                 6.0
race                           

In [47]:
train_data.df['age'].dtypes == 'int64'

True

In [36]:
train_data.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 40 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   age                                         199523 non-null  int64 
 1   class of worker                             199523 non-null  object
 2   detailed industry recode                    199523 non-null  int64 
 3   detailed occupation recode                  199523 non-null  int64 
 4   education                                   199523 non-null  object
 5   wage per hour                               199523 non-null  int64 
 6   enroll in edu inst last wk                  199523 non-null  object
 7   marital stat                                199523 non-null  object
 8   major industry code                         199523 non-null  object
 9   major occupation code                       199523 non-null  object
 10  race    

In [9]:
train_data.numerical_scaling()
train_data.df.head(3)

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,target,capital_change
0,1.725879,Not in universe,0,0,High school graduate,-0.201629,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,-0.949346,95,-1,-0.084396
1,1.053560,Self-employed-not incorporated,4,34,Some college but no degree,-0.201629,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,1.180804,94,-1,-0.084396
2,-0.739291,Not in universe,0,0,10th grade,-0.201629,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,-0.949346,95,-1,-0.084396


In [10]:
train_data.categorical_encoding()
train_data.df.head(3)

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,target,capital_change
0,1.725879,-0.981964,-0.982003,-0.982003,-0.922367,-0.201629,-0.867853,-0.936921,-0.982003,-0.982003,...,-0.870538,-0.874015,-0.874017,-0.884299,-0.876976,-0.838127,-0.949346,-0.868741,-1,-0.084396
1,1.053560,-0.741859,-0.821858,-0.851925,-0.871531,-0.201629,-0.867853,-0.832258,-0.821858,-0.818407,...,-0.870538,-0.874015,-0.874017,-0.884299,-0.876976,-0.838127,1.180804,-0.883018,-1,-0.084396
2,-0.739291,-0.981964,-0.982003,-0.982003,-0.983591,-0.201629,-0.998549,-0.974169,-0.982003,-0.982003,...,-0.953488,-0.943734,-0.926573,-0.884299,-0.876976,-0.838127,-0.949346,-0.868741,-1,-0.084396


In [11]:
train_data.split_X_y('train')

In [12]:
train_data.y_train

0        -1
1        -1
2        -1
3        -1
4        -1
         ..
199518   -1
199519   -1
199520   -1
199521   -1
199522   -1
Name: target, Length: 199523, dtype: int64

In [13]:
train_data.pickle_data(phase='train')

In [14]:
test_data = DataPrep('./census_data/census_income_test.csv')
test_data.apply_all(phase='test')